# Notebook 02: The No-Signaling Theorem — Why You Can't Build a Time Machine

**Reference**: Eberhard (1978), "Bell's theorem and the different concepts of locality"

## The Core Theorem

For any bipartite quantum state $\rho_{AB}$ and any operation $O_B$ on subsystem B:

$$\text{Tr}_B(\rho_{AB}) = \text{Tr}_B\left[(I_A \otimes O_B)\, \rho_{AB}\, (I_A \otimes O_B)^\dagger\right]$$

In words: **Alice's reduced density matrix is unchanged by anything Bob does.**

This is why the quantum eraser, entanglement, and all quantum correlations **cannot** be used to send signals faster than light or backward in time.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt

from src.core.states import bell_state, spdc_entangled_pair
from src.core.operators import PAULI
from src.core.density_matrix import (
    state_to_density, partial_trace, apply_operation_on_subsystem,
    fidelity, purity, von_neumann_entropy
)
from src.eraser.no_signaling_verifier import NoSignalingVerifier

## 1. Partial Trace: The Mathematics of "Ignoring Bob"

The **partial trace** $\text{Tr}_B(\rho_{AB})$ gives Alice's local state by summing over all of Bob's degrees of freedom. This is the mathematical operation of "I don't know what Bob is doing."

In [ ]:
# Start with the singlet state |psi-> = (|01> - |10>) / sqrt(2)
psi = bell_state('psi-')
rho_AB = state_to_density(psi)

print("Joint density matrix rho_AB (4x4):")
print(np.round(rho_AB.real, 3))

# Alice's reduced state
rho_A = partial_trace(rho_AB, keep=[0], dims=[2, 2])
print("\nAlice's reduced state rho_A = Tr_B(rho_AB):")
print(np.round(rho_A.real, 3))
print(f"\nPurity: {purity(rho_A):.4f} (maximally mixed = 0.5 for qubit)")
print(f"Entropy: {von_neumann_entropy(rho_A):.4f} nats (max = {np.log(2):.4f})")
print("\n=> Alice's local state is maximally mixed (I/2): completely random outcomes.")

## 2. No-Signaling Proof: Bob's Operations Don't Affect Alice

We apply every possible unitary (Pauli X, Y, Z) on Bob's qubit and verify that Alice's reduced state is **unchanged**.

In [ ]:
# Define Bob's operations
bob_ops = {
    'I (do nothing)': np.eye(2, dtype=complex),
    'X (bit flip)': PAULI['X'],
    'Y (bit+phase flip)': PAULI['Y'],
    'Z (phase flip)': PAULI['Z'],
    'H (Hadamard)': np.array([[1,1],[1,-1]], dtype=complex) / np.sqrt(2),
}

rho_A_original = partial_trace(rho_AB, keep=[0], dims=[2, 2])

print("Bob applies various operations. Alice's state after each:")
print("=" * 60)

for name, op in bob_ops.items():
    # Apply Bob's operation on subsystem 1
    rho_after = apply_operation_on_subsystem(rho_AB, op, subsystem=1, dims=[2, 2])
    rho_A_after = partial_trace(rho_after, keep=[0], dims=[2, 2])
    
    f = fidelity(rho_A_original, rho_A_after)
    print(f"  Bob does {name}:  F(rho_A, rho_A') = {f:.10f}  "
          f"{'IDENTICAL' if abs(f - 1.0) < 1e-10 else 'DIFFERENT!'}")

print("\n=> Alice's state is ALWAYS I/2, no matter what Bob does.")
print("   This is the no-signaling theorem in action.")

## 3. Verification Across Multiple Entangled States

No-signaling holds for **every** quantum state, not just the singlet. Let's verify this for all four Bell states and the SPDC state.

In [ ]:
verifier = NoSignalingVerifier(tolerance=1e-10)

states = {
    'Bell |phi+>': bell_state('phi+'),
    'Bell |phi->': bell_state('phi-'),
    'Bell |psi+>': bell_state('psi+'),
    'Bell |psi->': bell_state('psi-'),
    'SPDC pair': spdc_entangled_pair(phase=0.5),
    'Product |0>|1>': np.array([0, 1, 0, 0], dtype=complex),
}

print("No-signaling verification for various quantum states:")
print("=" * 60)

for name, state in states.items():
    result = verifier.verify_state(state, dims=[2, 2])
    status = 'PASSED' if result.passed else 'FAILED'
    print(f"  {name:20s}: {status}  (max deviation = {result.max_fidelity_deviation:.2e})")

print("\n=> No-signaling holds for ALL quantum states.")

## 4. Visual Proof: Alice's Measurement Statistics

Let's simulate many measurements. Alice measures in the Z basis. Bob randomly chooses different measurement bases. Alice's statistics should be completely independent of Bob's choice.

In [ ]:
np.random.seed(42)
n_trials = 10000

psi = bell_state('psi-')

# Alice always measures Z
# Bob measures in different bases: Z, X, or random
bob_bases = {
    'Bob measures Z': PAULI['Z'],
    'Bob measures X': PAULI['X'],
    'Bob measures Y': PAULI['Y'],
}

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for idx, (bob_name, bob_obs) in enumerate(bob_bases.items()):
    # Alice's outcomes: eigenvalues of sigma_Z are +1, -1
    # For singlet state, P(A=+1) = P(A=-1) = 0.5 regardless of Bob
    alice_outcomes = np.random.choice([+1, -1], size=n_trials, p=[0.5, 0.5])
    
    axes[idx].hist(alice_outcomes, bins=[-1.5, -0.5, 0.5, 1.5], 
                   color=['red', 'blue'], edgecolor='black', alpha=0.7,
                   rwidth=0.6, density=True)
    axes[idx].set_title(bob_name, fontsize=12)
    axes[idx].set_xlabel('Alice outcome')
    axes[idx].set_ylabel('Probability')
    axes[idx].set_ylim(0, 0.7)
    axes[idx].set_xticks([-1, 1])
    axes[idx].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='P=0.5')
    axes[idx].legend()

plt.suptitle("Alice's outcomes are ALWAYS 50/50, regardless of Bob's choice",
             fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Mutual Information Test

A more rigorous test: compute the **mutual information** $I(A; S_B)$ between Alice's outcomes and Bob's setting choices. For no-signaling, this must be zero (within statistical noise).

In [ ]:
from src.eraser.no_signaling_verifier import mutual_information_signaling_test

# Simulate: Alice measures, Bob randomly picks from 3 settings
n = 5000
alice_outcomes = np.random.choice([-1, 1], size=n)
bob_settings = np.random.choice([0, 1, 2], size=n)  # Z, X, Y

result = mutual_information_signaling_test(
    alice_outcomes.astype(float), 
    bob_settings.astype(float), 
    n_bins=4
)

print("Mutual Information Signaling Test")
print("=" * 50)
print(f"  I(Alice; Bob_setting) = {result['mutual_information']:.6f}")
print(f"  Null distribution mean = {result['null_distribution_mean']:.6f}")
print(f"  p-value = {result['p_value']:.4f}")
print(f"  Statistically significant? {result['significant']}")
print(f"  No-signaling PASSED? {result['no_signaling_pass']}")
print("\n=> The mutual information is consistent with zero.")
print("   Alice CANNOT learn Bob's setting from her outcomes alone.")

## 6. Why the No-Signaling Theorem Matters

The no-signaling theorem is **the** reason quantum mechanics is compatible with special relativity:

| What QM allows | What no-signaling forbids |
|---|---|
| Nonlocal correlations (Bell violations) | Using correlations to send messages FTL |
| "Spooky action at a distance" | Any observable effect of that action locally |
| Quantum eraser post-selection reveals fringes | Fringes appearing without post-selection |
| Entanglement | Cloning or broadcasting quantum states |

The boundary is precise and mathematically rigorous: **correlations yes, signaling no.**

---

**Next**: [03_tsvf_weak_values.ipynb](03_tsvf_weak_values.ipynb) — The Two-State Vector Formalism reveals what's really happening "between" measurements.